In [18]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [10]:
df = pd.read_csv('train.csv')
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [11]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [14]:
#IMPUTATION TRANSFORMER

trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]), #index used (instead of names) so that further process doesnt break in pipeline
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [15]:
#One hot encoding

trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')


In [16]:
#Scaling

trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [17]:
#Feature Selection

trf4 = SelectKBest(score_func = chi2 , k = 5)

In [19]:
#Train The Model

trf5 = DecisionTreeClassifier()

<h2>Create Pipeline</h2>

In [21]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5),
])

<h2>Pipeline v/s make_pipeline</h2>

<p>Pipeline requires naming of steps (like above) whereas make_pipeline does not.</p>
<p>(Same Thing applies to ColumnTransformer ans make_column_transformer)</p>

In [23]:
#Alternat Syntax

pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [24]:
#train 
pipe.fit(x_train,y_train)

C:\Users\shivam\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=5,
                             score_func=<function chi2 at 0x000001F49AD40E00>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

<h2>explore the Pipeline</h2>

In [26]:
#display pipeline 
from sklearn import set_config
set_config(display='diagram')

In [27]:
#predict

y_pred = pipe.predict(x_test)

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

<h2>Exporting the  Pipeline</h2>

In [29]:
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))